In [1]:
import pandas as pd
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Model
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
from tensorflow.keras.layers import ZeroPadding2D, BatchNormalization
from keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [2]:
import cv2
import os
import numpy as np
orginal_image = "E:\Saaswath\optical glcoma\SOURCE CODE\dataset\Glaucomatous"
glauucoma = os.listdir(orginal_image)

for i in glauucoma:
    orginal = cv2.imread(orginal_image + '/' +i)
    cv2.imshow('Orginal',orginal)

    mask = np.zeros(orginal.shape[:2],dtype="uint8")
    cv2.circle(mask,(145,200),100,255,-1)
    

    masked = cv2.bitwise_or(orginal,orginal,mask=mask)
    cv2.imshow("Mask Applied",masked)
    if cv2.waitKey(0) or ord('q'):
        cv2.destroyAllWindows()
        break

In [3]:
base_model = Sequential()

base_model.add(Conv2D(64, (9, 9), input_shape=(100, 100, 3), padding='same'))
base_model.add(BatchNormalization())
base_model.add(Activation('relu'))
base_model.add(MaxPooling2D(pool_size=(2, 2)))

base_model.add(Conv2D(128, (5, 5), padding='same'))
base_model.add(BatchNormalization())
base_model.add(Activation('relu'))
base_model.add(MaxPooling2D(pool_size=(2, 2)))

base_model.add(ZeroPadding2D((1, 1)))
base_model.add(Conv2D(256, (3, 3), padding='same'))
base_model.add(BatchNormalization())
base_model.add(Activation('relu'))
base_model.add(MaxPooling2D(pool_size=(2, 2)))

base_model.add(ZeroPadding2D((1, 1)))
base_model.add(Conv2D(512, (3, 3), padding='same'))
base_model.add(BatchNormalization())
base_model.add(Activation('relu'))

base_model.add(ZeroPadding2D((1, 1)))
base_model.add(Conv2D(512, (3, 3), padding='same'))
base_model.add(BatchNormalization())
base_model.add(Activation('relu'))
base_model.add(MaxPooling2D(pool_size=(2, 2)))

base_model.add(Flatten())
base_model.add(Dense(64))
base_model.add(BatchNormalization())
base_model.add(Activation('relu'))
base_model.add(Dropout(0.5))

base_model.add(Dense(128))
base_model.add(BatchNormalization())
base_model.add(Activation('relu'))
base_model.add(Dropout(0.5))

base_model.add(Dense(1))
base_model.add(BatchNormalization())
base_model.add(Activation('sigmoid'))

base_model.compile(loss='binary_crossentropy',
                   optimizer='adam', metrics=['accuracy'])

base_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 100, 100, 64)      15616     
                                                                 
 batch_normalization (BatchN  (None, 100, 100, 64)     256       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 100, 100, 64)      0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 50, 50, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 50, 50, 128)       204928    
                                                                 
 batch_normalization_1 (Batc  (None, 50, 50, 128)      5

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=30,
                                   shear_range=0.2,
                                   zoom_range=[0.8, 1.2],
                                   horizontal_flip=True,
                                   #                                     vertical_flip = True,
                                   fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale=1./255)

training_set = train_datagen.flow_from_directory('./dataset',
                                                 target_size=(100, 100),
                                                 batch_size=64,
                                                 class_mode='binary')

test_set = test_datagen.flow_from_directory('./dataset',
                                            target_size=(100, 100),
                                            batch_size=64,
                                            class_mode='binary')
my_callbacks = [
    # tf.keras.callbacks.EarlyStopping(patience=4, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(
        factor=0.1, patience=3, min_lr=0.00001, verbose=1),
    tf.keras.callbacks.ModelCheckpoint('my_model2.h5',
                                       verbose=1, save_best_only=True, save_weights_only=False)
]

Found 184 images belonging to 2 classes.
Found 184 images belonging to 2 classes.


In [5]:
base_model.fit(training_set, epochs=20,
               validation_data=test_set, callbacks=my_callbacks)

Epoch 1/20
3/3 [==============================] - ETA: 0s - loss: 0.6576 - accuracy: 0.6413 
Epoch 1: val_loss improved from inf to 0.68659, saving model to my_model2.h5
3/3 [==============================] - 44s 15s/step - loss: 0.6576 - accuracy: 0.6413 - val_loss: 0.6866 - val_accuracy: 0.5054 - lr: 0.0010
Epoch 2/20
3/3 [==============================] - ETA: 0s - loss: 0.6077 - accuracy: 0.6359
Epoch 2: val_loss did not improve from 0.68659
3/3 [==============================] - 40s 15s/step - loss: 0.6077 - accuracy: 0.6359 - val_loss: 0.7382 - val_accuracy: 0.5000 - lr: 0.0010
Epoch 3/20
3/3 [==============================] - ETA: 0s - loss: 0.6368 - accuracy: 0.6848 
Epoch 3: val_loss did not improve from 0.68659
3/3 [==============================] - 42s 15s/step - loss: 0.6368 - accuracy: 0.6848 - val_loss: 0.8250 - val_accuracy: 0.5109 - lr: 0.0010
Epoch 4/20
3/3 [==============================] - ETA: 0s - loss: 0.5540 - accuracy: 0.7554 
Epoch 4: val_loss improved from 0.6

In [6]:
from tensorflow.keras.preprocessing import image
data = pd.DataFrame()
for i, layer in enumerate(base_model.layers):
    model = Model(inputs=base_model.input,outputs=base_model.get_layer(layer.name).output)
    img_path = r'E:\Saaswath\optical glcoma\SOURCE CODE\dataset\Glaucomatous\glaucoma_389.jpg'
    img = image.load_img(img_path,target_size=(100,100))
    x = image.img_to_array(img)
    x = np.expand_dims(x,axis=0)
    pred = model.predict(x)
    reshape = np.reshape(pred, (-1))
    data["feature"] = pd.Series(reshape)
    data.to_csv("feature_extraction1.csv")